In [ ]:
from model_selection import *
from old_preprocesing import *
from tuning import *

In [ ]:
# Printing the best parameters for FastText

#X_train = load_dataset().pipe(extract_date_features).pipe(drop_duplicates).pipe(count_characters, trainset = True).pipe(clean_text).pipe(add_user_text).pipe(print_best_params)

In [3]:
filename = "./DSL2122_january_dataset/evaluation.csv"

X_train = load_dataset().pipe(extract_date_features).pipe(drop_duplicates).pipe(count_characters, trainset = True).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)
X_test = load_dataset(filepath=filename).pipe(extract_date_features).pipe(count_characters, trainset =False).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)

X_train, X_test = text_mining_tfdf(X_train, X_test, min_df=0.01) 

X_train, X_test = add_word_embeddings(X_train, X_test)

X_train, X_test = drop_columns(X_train, X_test=X_test)


print('Start Normalizing')
X_train, X_test, y_train = normalize(X_train, X_test)
print('Finish Normalizing')

100%|██████████| 74999/74999 [00:08<00:00, 8409.33it/s]
/home/christian/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 're', 've'] not in stop_words.
  warnings.warn(
/home/christian/.local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Sto facendo autovalidation


Read 3M words
Number of words:  196376
Number of labels: 2
Progress: 100.0% words/sec/thread:  111516 lr:  0.000000 avg.loss:  0.399960 ETA:   0h 0m 0s


Start Normalizing
Finish Normalizing


In [4]:
clf = RandomForestClassifier(max_features='sqrt', min_samples_leaf=20, n_estimators=500, random_state=42)
file_name = "prova_RF.csv"
print(f'Start Training with {clf}')
clf.fit(X_train,y_train)
print(f'Finish Training with {clf}')

y_pred = clf.predict(X_test)
pd.Series(y_pred, name="Predicted").to_csv(file_name, index_label="Id")
#save_results(y_pred, fp = file_name)
print(f"File Salvato con questo nome: {file_name}!\n\n\n")

Start Training with RandomForestClassifier()
Finish Training with RandomForestClassifier()
File Salvato con questo nome: prova_RF.csv!





In [ ]:
clf = HistGradientBoostingClassifier()
file_name = "prova_HGBC.csv"
print(f'Start Training with {clf}')
clf.fit(X_train,y_train)
print(f'Finish Training with {clf}')

y_pred = clf.predict(X_test)
pd.Series(y_pred, name="Predicted").to_csv(file_name, index_label="Id")
#save_results(y_pred, fp = file_name)
print(f"File Salvato con questo nome: {file_name}!\n\n\n")

In [5]:
# Split 80/20 %  the training set in order to tune the parameters

dataframe = load_dataset()
y = dataframe.pop('sentiment')
X = dataframe

print("Split dataset in train and test set")
X_train_hp, X_test_hp, y_train_hp, y_test = train_test_split(X, y, train_size= 0.8, test_size=0.2, random_state= 42)
X_train_hp = pd.concat([X_train_hp, y_train_hp], axis=1)

print("Start preprocessing")
X_train = X_train_hp.pipe(extract_date_features).pipe(drop_duplicates).pipe(count_characters, trainset = True).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)
X_test = X_test_hp.pipe(extract_date_features).pipe(count_characters, trainset =False).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)

X_train, X_test = text_mining_tfdf(X_train, X_test, min_df=0.01) 

X_train, X_test = add_word_embeddings(X_train, X_test)

X_train, X_test = drop_columns(X_train, X_test)

print('Start Normalizing')
X_train, X_test, y_train = normalize(X_train, X_test)
print('Finish Normalizing')

print("Finish preprocessing")

Split dataset in train and test set
Start preprocessing


/home/christian/Desktop/DLS-Project/old_preprocesing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['char_count'] = list(map(lambda x: len(html.unescape(x)), tweets['text']))
100%|██████████| 44999/44999 [00:05<00:00, 8343.61it/s]
/home/christian/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 're', 've'] not in stop_words.
  warnings.warn(
/home/christian/.local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead

Sto facendo autovalidation


Read 2M words
Number of words:  171588
Number of labels: 2
Progress: 100.0% words/sec/thread:  108336 lr:  0.000000 avg.loss:  0.415174 ETA:   0h 0m 0s


Start Normalizing
Finish Normalizing
Finish preprocessing


In [6]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

parameters_rf= {
    "max_features" : ["sqrt", "log2"], 
    "criterion": ["gini", "entropy"],
    "min_samples_leaf":[20, 35, 60],
}


clf= RandomForestClassifier(n_estimators= 500, random_state=42)


cv = HalvingGridSearchCV(clf,parameters_rf, cv=3, scoring="f1_macro", n_jobs=-1).fit(X_train,y_train)
print(cv.best_estimator_)
print(cv.best_score_)

RandomForestClassifier(max_features='sqrt', min_samples_leaf=20,
                       n_estimators=500, random_state=42)
0.8913604276971134


In [7]:
clf = HistGradientBoostingClassifier(random_state=42,loss= "binary_crossentropy")

parameters_HGBC ={
    "learning_rate":[ 0.01,0.05,0.1,1, 0.5],
    "max_leaf_nodes": [20,30,50,None],
    "min_samples_leaf":[ 2, 4, 10],
    "l2_regularization" : [0, 0.2, 0.1],
    "early_stopping": [True, False]

}

sh = HalvingGridSearchCV(clf, parameters_HGBC, cv=3, factor=2, min_resources= 1000, scoring='f1_macro').fit(X_train, y_train)
print(sh.best_estimator_)
print(sh.best_score_)

KeyboardInterrupt: 

In [ ]:
# Droppa le colonne tf-df

def test_models(X_train, X_test, y_train, y_test):
    models = [LinearSVC(), RandomForestClassifier(), BernoulliNB, HistGradientBoostingClassifier()]
    names= ["Linear SVC", "Random Forest", "Bernoulli Nayve-Bayes", "Hist GBC"]

    f1_scores = []
    for clf, name in zip(models, names):

        print(f'Start training model: {name}')

        start_time = time.time()
        f1_score = cross_val_score(clf, X, y, cv = 5, scoring = 'f1_macro')
        finish_time = time.time()

        print(f'Finishing training model: {name}, trained in {finish_time-start_time}')

        f1 = f1_score.mean()
        f1_scores.append(f1)

        print(f'Score of {name} model performed: {f1}\n\n\n')

    col1 = pd.Series(names)
    col2 = pd.Series(f1_scores)
            

    result = pd.concat([col1, col2], axis = 'columns')
    result.columns = ['Model Name', 'F1 Score']

    return result

In [ ]:
# Testa i modelli (TUTTE E TRE)

test_models(X_train, X_test, y_train, y_test)

#Droppa le colonne della tf_df(WE SENTYMENT)

drop_columns
X_train.drop(columns=drop_columns, inplace=True)
X_test.drop(columns=drop_columns, inplace=True)

test_models(X_train, X_test, y_train, y_test)

#Droppa colonne del sentiment(SOLO WE)

drop_cols = ["neg", "neu", "pos", "compound", "polarity", "subjectivity"]
X_train.drop(columns=drop_columns, inplace=True)
X_test.drop(columns=drop_columns, inplace=True)

test_models(X_train, X_test, y_train, y_test)


In [ ]:
from old_preprocesing import *

filename = "./DSL2122_january_dataset/evaluation.csv"

X_train = load_dataset().pipe(extract_date_features).pipe(drop_duplicates).pipe(count_characters, trainset = True).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)
X_test = load_dataset(filepath=filename).pipe(extract_date_features).pipe(count_characters, trainset =False).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)

X_train, X_test = text_mining_tfdf(X_train, X_test, min_df=0.01) 

X_train, X_test = add_word_embeddings(X_train, X_test)

X_train, X_test = drop_columns(X_train, X_test=X_test)


print('Start Normalizing')
X_train, X_test, y_train = normalize(X_train, X_test)
print('Finish Normalizing')


from sklearn.ensemble import HistGradientBoostingClassifier

clf = HistGradientBoostingClassifier(early_stopping=False, l2_regularization=0.1,
                               learning_rate=0.05, loss='binary_crossentropy',
                               max_leaf_nodes=20, min_samples_leaf=10,
                               random_state=42)
file_name = "prova_HGBC_vecchio.csv"
print(f'Start Training with {clf}')
clf.fit(X_train,y_train)
print(f'Finish Training with {clf}')

y_pred = clf.predict(X_test)
pd.Series(y_pred, name="Predicted").to_csv(file_name, index_label="Id")
#save_results(y_pred, fp = file_name)
print(f"File Salvato con questo nome: {file_name}!\n\n\n")

KeyboardInterrupt: 